In [202]:
!which jupyter

/home/ubuntu/thesis_env2/bin/jupyter


In [203]:
import pandas as pd
import numpy as np
import datetime
import spacy
import en_core_web_sm
#from spacy import displacy

In [204]:
df = pd.read_csv('./data/covid19_articles_20201231.csv')

# Date to datetime
df['date'] = pd.to_datetime(df['date'])

# Drop duplicates
num_articles_pre_drop = len(df)
df = df.drop_duplicates(subset='content')
print('no. articles dropped: ', num_articles_pre_drop - len(df))
print('no. of articles: ', len(df))

# Drop articles where text longer than 1,000,000 characters
df = df[df['content'].apply(lambda x: len(x) <= 1000000)].copy()
print('no. of articles after dropping long articles: ', len(df))

print(df.info())

no. articles dropped:  127
no. of articles:  368920
no. of articles after dropping long articles:  368918
<class 'pandas.core.frame.DataFrame'>
Int64Index: 368918 entries, 0 to 369046
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   author      181789 non-null  object        
 1   date        368918 non-null  datetime64[ns]
 2   domain      368918 non-null  object        
 3   title       368833 non-null  object        
 4   url         368918 non-null  object        
 5   content     368918 non-null  object        
 6   topic_area  368918 non-null  object        
dtypes: datetime64[ns](1), object(6)
memory usage: 22.5+ MB
None


In [ ]:
df.index

In [ ]:
# Get index numbers of articles to label
np.random.seed(777)
article_indices = np.random.choice(df.index, size=1000, replace=False)
article_indices

In [ ]:
# Load wip file
#file_latest = 'df_lab_2021_07_11_21_20.csv'
file_latest = 'df_lab_latest.csv'
df_lab = pd.read_csv('./data/' + file_latest, index_col=0)
print(sum(df_lab['org_names_listed'].isna()))

In [205]:
df_lab

,author,date,domain,title,url,content,topic_area,org_names_listed,org_names_listed_uo
272801,NaN,2020-08-15,finance.yahoo,Julen Lopetegui: We must be at our best to bea...,https://finance.yahoo.com/news/julen-lopetegui...,Head coach Julen Lopetegui knows Sevilla will ...,business,[],[]
200452,NaN,2020-07-11,finance.yahoo,Gallagher Premiership teams will not be punish...,https://uk.finance.yahoo.com/news/gallagher-pr...,Gallagher Premiership clubs will have flexibil...,business,[],[]
360615,NaN,2020-12-14,marketscreener,"Don't stockpile food, minister tells British s...",https://www.marketscreener.com/quote/currency/...,"Britain and the EU agreed on Sunday to ""go the...",business,['Tesco PLC'],['Tesco']
348240,Joe Hoppe,2020-11-23,marketwatch,"Wynnstay Properties profit halves, lifts dividend",https://www.marketwatch.com/story/wynnstay-pro...,Wynnstay Properties PLC said Monday that its f...,business,['Wynnstay Properties Plc'],['Wynnstay Properties PLC']
150912,NaN,2020-06-11,finance.yahoo,U.K. Scientists Defy Johnson to Speak Out on V...,https://ca.finance.yahoo.com/news/johnson-unde...,(Bloomberg) -- U.K. Prime Minister Boris Johns...,business,[],[]
...,...,...,...,...,...,...,...,...,...
169245,By Reuters,2020-06-24,nytimes,"With No Tourists to Watch Migration, Kenyan Op...",https://www.nytimes.com/reuters/2020/06/24/bus...,NAIROBI — For Kenyan-based safari operator Saf...,business,NaN,NaN
306948,NaN,2020-09-12,marketscreener,"Pfizer, BioNTech propose expanding COVID-19 va...",https://www.marketscreener.com/news/latest/Pfi...,Pfizer Inc and BioNTech SE\non Saturday propos...,business,NaN,NaN
111840,NaN,2020-05-13,marketscreener,Ameresco : Announces Commercial Operations of ...,https://www.marketscreener.com/AMERESCO-INC-64...,Beale Hill Wind Farm is the first renewable ge...,business,NaN,NaN
72139,NaN,2020-04-16,marketscreener,"L'Oréal: News release: ""First Quarter 2020 Sales""",https://www.marketscreener.com/quote/stock/L-O...,"Clichy, 16 April 2020 at 6:00 p.m. First Quart...",business,NaN,NaN


In [ ]:
ner_processor = en_core_web_sm.load()

In [ ]:
['AFP', 'Associated Press', 'BBC', 'Bloomberg', 'CNN', 'Daily Express', 'Ducker Frontier', 'Financial Times', 'Guardian', 'Marketwatch', 'NBC', 
 'Prudent Speculator', 'Reuters', 'RNZ', 'Sky News', 'SpreadEx', 'The Daily Mail', 'The Sun', 'The Canadian Press', 'LBC', 'New York Times'] # Reuters, New York Times also listed

['Aberdeen Standard Investments', 'Australian Bureau of Statistics', 'Commonwealth Financial Network', 'Federal Reserve', 'National Health Service', 
 'National Rail', 'New York Stock Exchange' 'NHS', 'Oxford University', 'Scientific Advisory Group for Emergencies', 'Securities and Exchange Commission', 
 'The Bank of England', 'The British Retail Consortium', 'The Office for Budget Responsibility', 'The Organisation for Economic Co-operation and Development', 
 'The Reserve Bank of Australia', 'The Vaccine Group', 'Toronto Stock Exchange', 'University of Plymouth', 'Eastspring Investments', 'NATO', 'FTSE', 'SEC', 
 'CDC', 'NYSE', 'C.D.C', 'W.H.O']


In [287]:
#df_lab[df_lab['content'].str.contains("Domino")]

In [288]:
#df_lab.loc[159684,'content']

In [671]:
num = 549
article_index = article_indices[num]
print(article_index)
print(df_lab.loc[article_index]['org_names_listed'])

article = df_lab.loc[article_index]['content']
spacy.displacy.render(ner_processor(article), style='ent', options = {"ents": ["ORG"]})

197122
nan


In [395]:
listed_orgs =    []
listed_orgs_uo = []

In [672]:
#article_index = 197122
listed_orgs =    []
listed_orgs_uo = []

df_lab.at[article_index,'org_names_listed'] = listed_orgs
df_lab.at[article_index,'org_names_listed_uo'] = listed_orgs_uo

In [673]:
df_lab.loc[article_indices[540:550]]

,author,date,domain,title,url,content,topic_area,org_names_listed,org_names_listed_uo
323452,https://www.theguardian.com/profile/johncrace,2020-09-28,theguardian,There may be a Covid crisis but Westminster st...,https://www.theguardian.com/politics/2020/sep/...,You might have thought the Commons would have ...,general,[],[]
347118,Alessandra Scotto di Santolo,2020-11-20,express,Scottish independence: Labour MP skewered over...,https://www.express.co.uk/news/politics/136248...,We will use your email address only for sendi...,general,[],[]
211436,NaN,2020-07-16,finance.yahoo,ALSTOM SA: Alstom Q1 2020/21 Orders and Sales,https://uk.finance.yahoo.com/news/alstom-sa-al...,Alstom Q1 2020/21 Orders and Sales 16 July 20...,business,"[Alstom SA, Bombardier Inc.]","[Alstom, Bombardier Inc.]"
70679,Berkeley Lovelace Jr.@BerkeleyJr,2020-04-16,cnbc,Gilead stock pops after report says coronaviru...,https://www.cnbc.com/2020/04/16/gilead-stock-s...,Gilead Sciences shares popped by more than 8% ...,finance,"[Gilead Sciences, Inc.]",[Gilead Sciences]
293956,NaN,2020-09-03,finance.yahoo,"South African healthcare workers protest, thre...",https://finance.yahoo.com/news/south-african-h...,"PRETORIA, South Africa (AP) — South African he...",business,[],[]
40782,Peter Hobson,2020-03-27,reuters,UPDATE 1- Coronavirus pushes London banks to c...,https://www.reuters.com/article/precious-refin...,"By Peter Hobson LONDON, March 27 (Reuters) - ...",business,"[HSBC Holdings plc, JPMorgan Chase & Co., UBS ...","[HSBC, JPMorgan, UBS]"
249444,NaN,2020-07-31,finance.yahoo,Outlook for U.S. value stocks uncertain with c...,https://sg.finance.yahoo.com/news/outlook-u-va...,- After a rally led by growth companies that ...,business,[Bank of America Corporation],[Bank of America]
107226,NaN,2020-05-11,marketscreener,Long-suffering Canadian oilpatch faces 'bigges...,https://www.marketscreener.com/news/Long-suffe...,By Jeff Lewis and Rod Nickel Canada's oil patc...,business,"[Suncor Energy Inc., Cenovus Energy Inc., Husk...","[Suncor Energy Inc, Cenovus Energy Inc, Husky ..."
13671,"Analysis by Laura He, CNN Business",2020-03-06,cnn,China has hundreds of stressed out banks. Coro...,https://www.cnn.com/2020/03/06/economy/china-b...,China's banking system was on shaky ground be...,general,[],[]
197122,NaN,2020-07-10,finance.yahoo,"Detectives uncover suspected £500,000 Covid-19...",https://uk.finance.yahoo.com/news/detectives-u...,Two men have been arrested over an alleged sca...,business,[],[]


In [551]:
#df_lab.loc[article_index]

# Articles not done - earnings call transcripts
[191712]

[191712]

In [552]:
date_label = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M")
file_name = 'df_lab_' + date_label + '.csv'
print(file_name)
df_lab.to_csv('./data/' + file_name)

df_lab_2021_07_13_18_21.csv


In [553]:
file_latest = 'df_lab_2021_07_13_18_21.csv' #499
df_check = pd.read_csv('./data/' + file_latest, index_col=0)
print(sum(df_check['org_names_listed'].isna()))
df_check

500


,author,date,domain,title,url,content,topic_area,org_names_listed,org_names_listed_uo
272801,NaN,2020-08-15,finance.yahoo,Julen Lopetegui: We must be at our best to bea...,https://finance.yahoo.com/news/julen-lopetegui...,Head coach Julen Lopetegui knows Sevilla will ...,business,[],[]
200452,NaN,2020-07-11,finance.yahoo,Gallagher Premiership teams will not be punish...,https://uk.finance.yahoo.com/news/gallagher-pr...,Gallagher Premiership clubs will have flexibil...,business,[],[]
360615,NaN,2020-12-14,marketscreener,"Don't stockpile food, minister tells British s...",https://www.marketscreener.com/quote/currency/...,"Britain and the EU agreed on Sunday to ""go the...",business,['Tesco PLC'],['Tesco']
348240,Joe Hoppe,2020-11-23,marketwatch,"Wynnstay Properties profit halves, lifts dividend",https://www.marketwatch.com/story/wynnstay-pro...,Wynnstay Properties PLC said Monday that its f...,business,['Wynnstay Properties Plc'],['Wynnstay Properties PLC']
150912,NaN,2020-06-11,finance.yahoo,U.K. Scientists Defy Johnson to Speak Out on V...,https://ca.finance.yahoo.com/news/johnson-unde...,(Bloomberg) -- U.K. Prime Minister Boris Johns...,business,[],[]
...,...,...,...,...,...,...,...,...,...
169245,By Reuters,2020-06-24,nytimes,"With No Tourists to Watch Migration, Kenyan Op...",https://www.nytimes.com/reuters/2020/06/24/bus...,NAIROBI — For Kenyan-based safari operator Saf...,business,NaN,NaN
306948,NaN,2020-09-12,marketscreener,"Pfizer, BioNTech propose expanding COVID-19 va...",https://www.marketscreener.com/news/latest/Pfi...,Pfizer Inc and BioNTech SE\non Saturday propos...,business,NaN,NaN
111840,NaN,2020-05-13,marketscreener,Ameresco : Announces Commercial Operations of ...,https://www.marketscreener.com/AMERESCO-INC-64...,Beale Hill Wind Farm is the first renewable ge...,business,NaN,NaN
72139,NaN,2020-04-16,marketscreener,"L'Oréal: News release: ""First Quarter 2020 Sales""",https://www.marketscreener.com/quote/stock/L-O...,"Clichy, 16 April 2020 at 6:00 p.m. First Quart...",business,NaN,NaN


In [554]:
df_lab_latest = df_lab.copy()
df_lab_latest.to_csv('./data/df_lab_latest.csv')